This script extracts all satellite names from each country database (obtained from Space Explorer) with the satellite names obtained from the administration letters.
It produces two lists: found and not found (in the SE database).  It uses a soft-match string search and produces a best-match   

In [6]:
import pandas as pd
import numpy as np
import os
from IPython.display import display
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
pd.set_option('display.max_rows', None)

In [7]:
countries_folder = './countriestables'
# load all tables from the folders into a single pd file
# Initialize an empty list to store DataFrames
dataframes = []

# Loop through all files in the folder
for filename in os.listdir(countries_folder):
    if filename.endswith('.csv'):  # Assuming the tables are in CSV format
        filepath = os.path.join(countries_folder, filename)
        df = pd.read_csv(filepath)
        dataframes.append(df)

# Concatenate all DataFrames into a single DataFrame
combined_df = pd.concat(dataframes, ignore_index=True)

# Display the combined DataFrame
display(combined_df.head())
# print all the column names
print(combined_df.columns)

C:\Users\smen851\AppData\Local\Temp\ipykernel_20448\3508078801.py:10: DtypeWarning: Columns (22) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(filepath)
C:\Users\smen851\AppData\Local\Temp\ipykernel_20448\3508078801.py:10: DtypeWarning: Columns (22) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(filepath)
C:\Users\smen851\AppData\Local\Temp\ipykernel_20448\3508078801.py:10: DtypeWarning: Columns (20,22,23,30,31) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(filepath)
C:\Users\smen851\AppData\Local\Temp\ipykernel_20448\3508078801.py:10: DtypeWarning: Columns (3,20,22,23) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(filepath)
C:\Users\smen851\AppData\Local\Temp\ipykernel_20448\3508078801.py:10: DtypeWarning: Columns (23) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd

,com_el.ntc_id,com_el.tgt_ntc_id,com_el.adm,com_el.ntwk_org,com_el.sat_name,com_el.prov,com_el.d_rcv,com_el.st_cur,orbit.orb_id,orbit.nbr_sat_pl,...,emiss.seq_no,emiss.pwr_ds_max,emiss.design_emi,carrier_fr.freq_carr,carrier_fr.seq_emiss,carrier_fr.seq_no,srv_area.ctry,srv_area.f_excl_api,emiss_seq_no,carrier_fr_seq_no
0,117545401,NaN,AFS,NaN,ZACUBE-2,NaN,9.1/IA,15/12/2017 12:00:00 am,50,1.0,...,1.0,-65.0,6M40G2DDN,2225.0,1.0,1.0,AFG,N,NaN,NaN
1,117545401,NaN,AFS,NaN,ZACUBE-2,NaN,9.1/IA,15/12/2017 12:00:00 am,50,1.0,...,1.0,-37.0,9K50F1DBN,401.5,1.0,1.0,AFG,N,NaN,NaN
2,120545101,NaN,AFS,NaN,MDASAT-1,NaN,9.1/IA,13/05/2020 12:00:00 am,50,1.0,...,1.0,-68.0,6M40G1DDN,2245.0,1.0,1.0,AFS,N,NaN,NaN
3,120545101,NaN,AFS,NaN,MDASAT-1,NaN,9.1/IA,13/05/2020 12:00:00 am,50,1.0,...,1.0,-68.0,6M40G1DDN,2245.0,1.0,1.0,E,N,NaN,NaN
4,120545101,NaN,AFS,NaN,MDASAT-1,NaN,9.1/IA,13/05/2020 12:00:00 am,50,1.0,...,1.0,-68.0,6M40G1DDN,2245.0,1.0,1.0,I,N,NaN,NaN


Index(['com_el.ntc_id', ' com_el.tgt_ntc_id', ' com_el.adm',
       ' com_el.ntwk_org', ' com_el.sat_name', ' com_el.prov', ' com_el.d_rcv',
       ' com_el.st_cur', ' orbit.orb_id', ' orbit.nbr_sat_pl',
       ' orbit.inclin_ang', ' orbit.apog_km', ' orbit.perig_km',
       ' orbit.op_ht_km', ' s_beam.emi_rcp', ' s_beam.beam_name',
       ' grp.grp_id', ' grp.freq_min', ' grp.freq_max', ' grp.bdwdth',
       ' grp.d_inuse', ' grp.d_reg_limit', ' grp.d_prot_eff', ' grp.f_biu',
       ' emiss.seq_no', ' emiss.pwr_ds_max', ' emiss.design_emi',
       ' carrier_fr.freq_carr', ' carrier_fr.seq_emiss', ' carrier_fr.seq_no',
       ' srv_area.ctry', ' srv_area.f_excl_api', 'emiss_seq_no',
       'carrier_fr_seq_no'],
      dtype='object')


In [8]:

# Get all unique names in the column 'com_el.sat_name' and their first corresponding 'com_el.adm' column
unique_names_with_adm = combined_df.drop_duplicates(subset=[' com_el.sat_name'])[[' com_el.sat_name', ' com_el.adm']]
sat_names_db = unique_names_with_adm[' com_el.sat_name'].tolist()
adm_names_db = unique_names_with_adm[' com_el.adm'].tolist()
# Print the unique names with their corresponding 'com_el.adm' column without truncation
pd.set_option('display.max_rows', None)
display(unique_names_with_adm)


,com_el.sat_name,com_el.adm
0,ZACUBE-2,AFS
2,MDASAT-1,AFS
9,EOS AGRISAT-1,AFS
27,ALSAT-1B,ALG
35,ALCOMSAT-33.5W,ALG
37,ALSAT-3,ALG
277,ALEPH-1-E,ARG
280,ALEPH-3-X1,ARG
350,ALEPH-3-X2,ARG
406,ALEPH-3-X3,ARG


get the reference names from the satellitenames folder

In [9]:
names_folder = './satellitenames'
# get a list of names and a list of countries (name of the file dropping.txt)

# Initialize lists to store names and countries
names = []
countries = []

# Loop through all files in the folder
for filename in os.listdir(names_folder):
    if filename.endswith('.txt'):
        filepath = os.path.join(names_folder, filename)
        with open(filepath, 'r') as file:
            content = file.read().strip()
            if content:
                # Split the content by commas and extend the names list
                names.extend(content.split(', '))
                # Add the country name (filename without extension) to the countries list
                countries.extend([filename.split('.')[0]] * len(content.split(', ')))

# # Print the lists of names and countries
# print("Names:", names)
# print("Countries:", countries)


In [10]:
# Assuming unique_names_with_adm and names are already defined
# unique_names_with_adm is a DataFrame with columns 'com_el.sat_name' and 'com_el.adm'
# names is a list of satellite names

# Initialize lists to store the best match names and scores
best_match_names = []
scores = []

# Iterate over each name in the names list
for name in names:
    # Find the best match for the current name in the 'com_el.sat_name' column
    best_match, score = process.extractOne(name, sat_names_db)
    
    # If no match is found, set the best match to 'not found'
    if score < 95:  # You can adjust the threshold as needed
        best_match = 'not found'
    
    # Append the best match and score to the respective lists
    best_match_names.append(best_match)
    scores.append(score)

result_df = pd.DataFrame({
    'administration': countries,
    'sat_name': names,
    'best_match': best_match_names,
    'score': scores
})

# Export the DataFrame as a CSV file
result_df.to_csv('satellite_matches.csv', index=False)

print("The DataFrame has been exported as 'satellite_matches.csv'.")

# # Print the results
# print("Best Match Names:", best_match_names)
# print("Scores:", scores)
# make a pd dataframe with 4 columns administration, sat_name, best_match, score and export as csv

The DataFrame has been exported as 'satellite_matches.csv'.
